In [1]:
import os
import pandas as pd
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
import csv
from pathlib import Path
from scipy import signal
import matplotlib.pyplot as plt

import torch.nn.functional as F


path = os.path.join(os.getcwd(),"datasets","Longer")

def files(path, ext=None):
    for (dirpath, dirnames, filenames) in os.walk(path):
        for file in filenames:
            if os.path.isfile(os.path.join(dirpath, file)):
                f_dta = None
                for file_dta in filenames:
                    if file.split(".")[0] in file_dta and file != file_dta:
                        f_dta = file_dta
                
                if ext is None:
                    yield dirpath, file, f_dta
                elif ext in file.split(".")[-1]:
                    yield dirpath, file, f_dta


file_paths = []
for file in files(path, "csv"):
    file_paths.append(file)

In [2]:

def read_from_csv_with_datetime(file_path):
    with open(file_path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        headers = next(reader)
        hd_time = [hd for hd in headers if 'datetime' in hd.lower()]
        hd_icp = [hd for hd in headers if 'icp' in hd.lower()]
    with open(file_path) as csv_file:
        raw_data = pd.read_csv(csv_file, sep=',', usecols=[hd_time[0], hd_icp[0]])

    time = raw_data[hd_time].to_numpy()
    t0 = (time[0] - int(time[0])) * 24 * 3600
    raw_t = (time - np.floor(time)) * 24 * 3600 - t0
    raw_icp = raw_data[hd_icp].to_numpy()

    t = np.delete(raw_t, np.where(np.isnan(raw_icp)))
    icp = np.delete(raw_icp, np.where(np.isnan(raw_icp)))
    fs_hat = round(1/(t[1]-t[0]), -1)

    return t, icp, fs_hat


def detect_icp_pulses(icp_raw, fs=100):
    icp = signal.detrend(icp_raw)
    Wc1 = 20 / fs
    Wc2 = 0.7 / (fs/2)

    b1, a1 = signal.iirfilter(N=8, Wn=Wc1, btype='lowpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b1, a1, icp)

    b2, a2 = signal.iirfilter(N=8, Wn=Wc2, btype='highpass', rs=60, rp=1, ftype='cheby1', output='ba')
    f_icp = signal.filtfilt(b2, a2, f_icp)

    dICP = np.diff(f_icp)
    SSF = np.insert(dICP, 0, 0)
    SSF[np.argwhere(SSF < 0)] = 0

    z_icp = np.copy(f_icp)
    z_icp[np.argwhere(z_icp < 0)] = 0
    nSSF = np.multiply(SSF, z_icp)

    min_dist = 0.4 * fs
    pk_locs = signal.find_peaks(nSSF, distance=min_dist)[0]

    po_locs = []
    for pk_ind, pk_loc in enumerate(pk_locs[:-1]):
        w = f_icp[pk_locs[pk_ind]:pk_locs[pk_ind + 1]]
        w_rev = w[::-1]
        offset = np.argmin(w_rev)
        po_locs.append(pk_locs[pk_ind + 1] - offset - 1)

    return f_icp, po_locs

# p = Path(r'test.csv')
# t, icp, fs = read_from_csv_with_datetime(p)
# f_icp, po_locs = detect_icp_pulses(icp, fs)

In [3]:
import os
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
is_odenet = True

device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)


class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.norm1 = norm(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.norm2 = norm(planes)
        self.conv2 = conv3x3(planes, planes)

    def forward(self, x):
        shortcut = x

        out = self.relu(self.norm1(x))

        if self.downsample is not None:
            shortcut = self.downsample(out)

        out = self.conv1(out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(out)

        return out + shortcut


class ConcatConv2d(nn.Module):

    def __init__(self, dim_in, dim_out, ksize=3, stride=1, padding=0, dilation=1, groups=1, bias=True, transpose=False):
        super(ConcatConv2d, self).__init__()
        module = nn.ConvTranspose1d if transpose else nn.Conv1d
        self._layer = module(
            dim_in + 1, dim_out, kernel_size=ksize, stride=stride, padding=padding, dilation=dilation, groups=groups,
            bias=bias
        )

    def forward(self, t, x):
        tt = torch.ones_like(x[:, :1, :]) * t
        ttx = torch.cat([tt, x], 1)
        return self._layer(ttx)


class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()
        self.norm1 = norm(dim)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm2 = norm(dim)
        self.conv2 = ConcatConv2d(dim, dim, 3, 1, 1)
        self.norm3 = norm(dim)
        self.nfe = 0

    def forward(self, t, x):
        self.nfe += 1
        out = self.norm1(x)
        out = self.relu(out)
        out = self.conv1(t, out)
        out = self.norm2(out)
        out = self.relu(out)
        out = self.conv2(t, out)
        out = self.norm3(out)
        return out


class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=1e-5, atol=1e-5)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value


class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)


downsampling_layers = [
        nn.Conv1d(1, 64, 3, 1),
        ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
        ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
    ]

feature_layers = [ODEBlock(ODEfunc(64))] if is_odenet else [ResBlock(64, 64) for _ in range(6)]
fc_layers = [norm(64), nn.ReLU(inplace=True), nn.AdaptiveAvgPool1d(1), Flatten(), nn.Dropout(0.6), nn.Linear(64, 5)]

model = nn.Sequential(*downsampling_layers, *feature_layers, *fc_layers).to(device)

In [6]:
from tqdm import tqdm
import pickle
from scipy.signal import blackmanharris
from scipy.stats import pearsonr


device = torch.device('cuda:' + str(0) if torch.cuda.is_available() else 'cpu')

def get_fourier_coeff(y):
    fft = np.fft.fft(y, n=180)/len(y)
    fft *= 2
    out = []
    out.append(fft[0].real)
    out += fft[1:-1].real
    out += -fft[1:-1].imag
    return out

def transform_pulse(pulse):
    fourier = get_fourier_coeff(pulse)
    return (fourier-min(fourier))/(max(fourier)-min(fourier))
    

def amp_from_fft(sig):
    windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(windowed)
    amplitudes = 2 / len(sig) * np.abs(f)
    i = np.argmax(amplitudes)
    return amplitudes[i]

def get_batches(pulses, batch_size=2048, sig_length=180):
    for i in range(0,len(pulses),batch_size):
        tensors = []
        for pulse in pulses[i:i+batch_size]:
            pulse = np.array(pulse)
            if len(pulse) <= sig_length:
                bckg = np.zeros(sig_length)
                bckg[-len(pulse):] = pulse
            else:
                middle = int(len(pulse)/2)
                bckg = pulse[middle-int(sig_length/2):middle+int(sig_length/2)]
            if len(bckg) != sig_length:
                print(len(bckg))
            tensors.append(bckg)
        yield torch.tensor(tensors,dtype=torch.double, device=device)

# model_path = os.path.join(os.getcwd(), "experiments", "2020-03-17_ODE_5cls_1", "model_1.pth")
# model.load_state_dict(torch.load(model_path, map_location=device)['state_dict'])
# model.eval()
model_path = os.path.join(os.getcwd(), "experiments", "NormalizedFourier_CNN_5cls_weighted_1", "model_weights", "model_final.pth")
model = torch.load(model_path, map_location=device)
    
for paths in tqdm(file_paths):
#     paths = file_paths[1]
    csv_file = read_from_csv_with_datetime(os.path.join(paths[0], paths[1]))
    f_icp, po_locs = detect_icp_pulses(csv_file[1], csv_file[2])
    pulses = [f_icp[po_locs[i]: po_locs[i+1]] for i in range(len(po_locs)-1)]
    pulses_f = [get_fourier_coeff(pulse) for pulse in pulses]
    pulses_f = (pulses_f - np.min(np.min(pulses_f)))/(np.max(np.max(pulses_f))- np.min(np.min(pulses_f)))
    all_outputs = []
    for batch in get_batches(pulses, sig_length=178):
#         batch = batch.unsqueeze(1)
        outputs = model(batch)
        softmax_outputs = F.softmax(outputs, dim=1).tolist()
        all_outputs += np.argmax(softmax_outputs, axis=1).tolist()
    diction = {
    "all_outputs": all_outputs}
    
    with open(os.path.join(os.getcwd(), "results","CNN",paths[1].split(".")[0]+".pkl" ), 'wb') as infile:
        loaded = pickle.dump(diction, infile)

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [23:38<00:00, 13.01s/it]


In [9]:
print(paths[1])

PAC10_1.csv


In [46]:
import copy
y = copy.deepcopy(all_outputs)
x = [int((po_locs[i]+ po_locs[i+1])/2) for i in range(len(po_locs)-1)]

In [35]:
cls1 = [1 if i == 0 else 0 for i in y ]
cls2 = [1 if i == 1 else 0 for i in y ]
cls3 = [1 if i == 2 else 0 for i in y ]
cls4 = [1 if i == 3 else 0 for i in y ]
cls5 = [1 if i == 4 else 0 for i in y ]

In [47]:
time = [csv_file[0][i] for i in x]

In [58]:
import json
diction = {
    "model_predictions": y,
    "model_idx": x,
    "model_time": time,
    "time": csv_file[0],
    "icp": csv_file[1],
    "fs_hat": csv_file[2],
    "f_icp": f_icp,
    "po_locs": po_locs,
    "pulses": pulses
}


NameError: name 'data' is not defined

In [2]:
import pickle
with open(os.path.join(os.getcwd(), "results",'PAC_1.pkl' ), 'rb') as infile:
    loaded = pickle.load(infile)

In [3]:
loaded.keys()

dict_keys(['model_predictions', 'model_idx', 'model_time', 'time', 'icp', 'fs_hat', 'f_icp', 'po_locs', 'pulses'])

In [14]:
from scipy.signal import blackmanharris
from scipy.stats import pearsonr

def amp_from_fft(sig):
    windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(windowed)
    amplitudes = 2 / len(sig) * np.abs(f)
    i = np.argmax(amplitudes)
    return amplitudes[i]

time_diff = 300*int(loaded["fs_hat"])
small_time_diff = 10*int(loaded["fs_hat"])
means = []
four = []
for j in range(0, len(loaded["icp"]), small_time_diff):
    means.append(np.mean(loaded["icp"][j:j+small_time_diff]))
    four.append(amp_from_fft(loaded["icp"][j:j+small_time_diff]))
    
raps = []
tms = []
y = loaded['model_predictions']
cls1 = [1 if i == 0 else 0 for i in y ]
cls2 = [1 if i == 1 else 0 for i in y ]
cls3 = [1 if i == 2 else 0 for i in y ]
cls4 = [1 if i == 3 else 0 for i in y ]
cls5 = [1 if i == 4 else 0 for i in y ]

m1=[]
m2=[]
m3=[]
m4=[]
m5=[]
po_locs = loaded["po_locs"]
model_times = [loaded["time"][int(po_locs[i+1])] for i in range(len(po_locs)-1)]
for i in range(30, len(means), 6):
    raps.append(pearsonr(means[i-30:i], four[i-30:i])[0])
    time_start = loaded["time"][(i-30)*small_time_diff]
    time_end = loaded["time"][i*small_time_diff]
    times = [1 if time_start<=t<time_end else 0 for t in model_times]
    m1.append(sum([cls1[i] if times[i]==1 else 0 for i in range(len(cls1))])/sum(times))
    m2.append(sum([cls2[i] if times[i]==1 else 0 for i in range(len(cls2))])/sum(times))
    m3.append(sum([cls3[i] if times[i]==1 else 0 for i in range(len(cls3))])/sum(times))
    m4.append(sum([cls4[i] if times[i]==1 else 0 for i in range(len(cls4))])/sum(times))
    m5.append(sum([cls5[i] if times[i]==1 else 0 for i in range(len(cls5))])/sum(times))
    tms.append(time_end)

In [18]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=tms, y=raps,
                    mode='lines',
                    name='Rap'))
fig.add_trace(go.Scatter(x=tms, y=m1,
                    mode='lines',
                    name='T1'))
fig.add_trace(go.Scatter(x=tms, y=m2,
                    mode='lines',
                    name='T2'))
fig.add_trace(go.Scatter(x=tms, y=m3,
                    mode='lines',
                    name='T3'))
fig.add_trace(go.Scatter(x=tms, y=m4,
                    mode='lines',
                    name='T4'))
fig.add_trace(go.Scatter(x=tms, y=m5,
                    mode='lines',
                    name='A+E'))
fig.update_layout(title='Changes in Rap value and percentages of predicted classes',
                   xaxis_title='Time[seconds]')

fig.show()

In [10]:
from tqdm import tqdm
import pickle
from scipy.signal import blackmanharris
from scipy.stats import pearsonr

def amp_from_fft(sig):
    windowed = sig * blackmanharris(len(sig))
    f = np.fft.fft(windowed)
    amplitudes = 2 / len(sig) * np.abs(f)
    i = np.argmax(amplitudes)
    return amplitudes[i]

def get_batches(pulses, batch_size=2048, sig_length=180):
    for i in range(0,len(pulses),batch_size):
        tensors = []
        for pulse in pulses[i:i+batch_size]:
            pulse = np.array(pulse)
            if len(pulse) <= sig_length:
                bckg = np.zeros(sig_length)
                bckg[-len(pulse):] = pulse
            else:
                middle = int(len(pulse)/2)
                bckg = pulse[middle-int(sig_length/2):middle+int(sig_length/2)]
            if len(bckg) != sig_length:
                print(len(bckg))
            tensors.append(bckg)
        yield torch.tensor(tensors,dtype=torch.float, device=device)

model_path = os.path.join(os.getcwd(), "experiments", "2020-03-17_ODE_5cls_1", "model_1.pth")
model.load_state_dict(torch.load(model_path, map_location=device)['state_dict'])
model.eval()

for paths in tqdm(file_paths):
    csv_file = read_from_csv_with_datetime(os.path.join(paths[0], paths[1]))
    f_icp, po_locs = detect_icp_pulses(csv_file[1], csv_file[2])

    model_times = [csv_file[0][int(po_locs[i+1])] for i in range(len(po_locs)-1)]
    
    print(len(all_outputs), len(model_times))
    diction = {
    "po_locs": po_locs,
    "model_times": model_times,
    }
    with open(os.path.join(os.getcwd(), "results","times",paths[1].split(".")[0]+".pkl" ), 'wb') as infile:
        loaded = pickle.dump(diction, infile)





  0%|                                                                                          | 0/109 [00:00<?, ?it/s]

144939 45722






  1%|▊                                                                                 | 1/109 [00:01<02:38,  1.47s/it]

144939 144939






  2%|█▌                                                                                | 2/109 [00:11<07:09,  4.01s/it]

...





 98%|██████████████████████████████████████████████████████████████████████████████▌ | 107/109 [07:19<00:05,  3.00s/it]

144939 105067






 99%|███████████████████████████████████████████████████████████████████████████████▎| 108/109 [07:22<00:03,  3.10s/it]

144939 109372






100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [07:26<00:00,  4.09s/it]
